<a href="https://colab.research.google.com/github/JairusTheAnalyst/JairusTheAnalyst/blob/main/ARM_in_Retail_Data_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

print("=== KENYAN RETAIL REAL-TIME & CONTEXT-AWARE ARM DEMONSTRATION ===\n")

# 1. Generate Kenyan Retail Dataset with Context Features
def generate_kenyan_retail_data(n_transactions=5000):
    """Generate supermarket transaction data with Kenyan context"""

    kenyan_products = {
        'staples': ['Maize Flour', 'Wheat Flour', 'Rice', 'Sugar', 'Cooking Oil', 'Salt'],
        'vegetables': ['Sukuma Wiki', 'Tomatoes', 'Onions', 'Potatoes', 'Cabbage', 'Carrots'],
        'proteins': ['Beef', 'Chicken', 'Eggs', 'Beans', 'Lentils', 'Milk'],
        'beverages': ['Chai Masala', 'Coffee', 'Soda', 'Juice', 'Water', 'Uji Mix'],
        'snacks': ['Biscuits', 'Crisps', 'Chocolate', 'Cakes', 'Bread', 'Mandazi']
    }

    contexts = {
        'weather': ['sunny', 'rainy', 'hot', 'cold'],
        'time_of_day': ['morning', 'afternoon', 'evening'],
        'season': ['dry', 'rainy', 'holiday'],
        'location': ['Nairobi', 'Mombasa', 'Kisumu', 'Nakuru', 'Eldoret']
    }

    transactions = []

    for i in range(n_transactions):
        transaction_id = f"TXN_{i+1:05d}"

        # Random context
        weather = random.choice(contexts['weather'])
        time_of_day = random.choice(contexts['time_of_day'])
        season = random.choice(contexts['season'])
        location = random.choice(contexts['location'])

        # Generate items based on context
        items = []

        # Weather-based items
        if weather == 'rainy':
            items.extend(random.sample(['Umbrella', 'Hot Chocolate', 'Soup'], random.randint(0, 2)))
        elif weather == 'hot':
            items.extend(random.sample(['Ice Cream', 'Cold Soda', 'Water'], random.randint(0, 2)))

        # Time-based items
        if time_of_day == 'morning':
            items.extend(random.sample(kenyan_products['beverages'][:3] + ['Bread', 'Eggs', 'Milk'], random.randint(1, 3)))
        elif time_of_day == 'evening':
            items.extend(random.sample(kenyan_products['vegetables'] + kenyan_products['proteins'][:3], random.randint(1, 3)))

        # Season-based items
        if season == 'rainy':
            items.extend(random.sample(['Rain Coat', 'Boots', 'Hot Beverages'], random.randint(0, 2)))
        elif season == 'holiday':
            items.extend(random.sample(['Christmas Decorations', 'Special Rice', 'Chicken'], random.randint(0, 2)))

        # Location-based items
        if location == 'Mombasa':
            items.extend(random.sample(['Coconut', 'Seafood', 'Swimwear'], random.randint(0, 2)))
        elif location == 'Nakuru':
            items.extend(random.sample(['Farm Vegetables', 'Fresh Milk', 'Cheese'], random.randint(0, 2)))

        # Always add some random staple items
        staples = random.sample(kenyan_products['staples'], random.randint(1, 3))
        items.extend(staples)

        # Remove duplicates and ensure minimum items
        items = list(set(items))
        if len(items) < 2:
            items.extend(random.sample(kenyan_products['staples'], 1))

        transactions.append({
            'transaction_id': transaction_id,
            'timestamp': datetime.now() - timedelta(hours=random.randint(0, 720)),
            'items': items,
            'weather': weather,
            'time_of_day': time_of_day,
            'season': season,
            'location': location,
            'total_items': len(items)
        })

    return pd.DataFrame(transactions)

# Generate data
print("1. Generating Kenyan Retail Dataset with Context...")
df = generate_kenyan_retail_data()
print(f"Generated {len(df)} transactions")
print(f"Sample transactions with context:")
print(df[['transaction_id', 'items', 'weather', 'time_of_day', 'location']].head(3).to_string(index=False))

# 2. Real-Time ARM Simulation
print("\n" + "="*50)
print("2. REAL-TIME ARM SIMULATION")
print("="*50)

class RealTimeARM:
    def __init__(self):
        self.transaction_buffer = []
        self.frequent_patterns = {}

    def process_transaction(self, transaction):
        """Process single transaction in real-time"""
        self.transaction_buffer.append(transaction)

        # Keep only recent transactions (sliding window)
        if len(self.transaction_buffer) > 1000:
            self.transaction_buffer.pop(0)

        # Update frequent patterns (simplified)
        self._update_patterns(transaction)

    def _update_patterns(self, transaction):
        """Update frequent patterns based on new transaction"""
        items = transaction['items']

        # Generate item pairs
        for i in range(len(items)):
            for j in range(i+1, len(items)):
                pair = tuple(sorted([items[i], items[j]]))
                self.frequent_patterns[pair] = self.frequent_patterns.get(pair, 0) + 1

    def get_recommendations(self, current_items, context):
        """Get real-time recommendations based on current items and context"""
        recommendations = {}

        for item in current_items:
            for pattern, count in self.frequent_patterns.items():
                if item in pattern:
                    other_item = pattern[1] if pattern[0] == item else pattern[0]

                    # Context-aware filtering
                    if self._is_context_relevant(other_item, context):
                        recommendations[other_item] = recommendations.get(other_item, 0) + count

        return sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:5]

    def _is_context_relevant(self, item, context):
        """Check if item is relevant to current context"""
        weather, time_of_day, location = context

        # Weather-based relevance
        if weather == 'rainy' and item in ['Umbrella', 'Hot Chocolate', 'Soup']:
            return True
        elif weather == 'hot' and item in ['Ice Cream', 'Cold Soda', 'Water']:
            return True

        # Time-based relevance
        if time_of_day == 'morning' and item in ['Bread', 'Eggs', 'Milk', 'Chai Masala']:
            return True
        elif time_of_day == 'evening' and item in ['Vegetables', 'Meat', 'Cooking Oil']:
            return True

        # Location-based relevance
        if location == 'Mombasa' and item in ['Coconut', 'Seafood']:
            return True

        return False

# Simulate real-time processing
print("\nSimulating real-time transaction processing...")
real_time_arm = RealTimeARM()

# Process some transactions
for _, transaction in df.head(100).iterrows():
    real_time_arm.process_transaction(transaction)

# Test real-time recommendations
print("\nTesting real-time recommendations:")
test_context = ('rainy', 'morning', 'Nairobi')
test_items = ['Maize Flour']

recommendations = real_time_arm.get_recommendations(test_items, test_context)
print(f"Context: Weather={test_context[0]}, Time={test_context[1]}, Location={test_context[2]}")
print(f"Current items: {test_items}")
print("Recommended items:")
for item, score in recommendations:
    print(f"  - {item} (score: {score})")

# 3. Context-Aware Pattern Mining
print("\n" + "="*50)
print("3. CONTEXT-AWARE PATTERN MINING")
print("="*50)

def mine_context_aware_patterns(df, context_condition):
    """Mine patterns specific to given context"""
    context_data = df[
        (df['weather'] == context_condition['weather']) &
        (df['time_of_day'] == context_condition['time_of_day']) &
        (df['location'] == context_condition['location'])
    ]

    print(f"\nMining patterns for context: {context_condition}")
    print(f"Transactions in this context: {len(context_data)}")

    # Count item co-occurrences in this context
    item_pairs = {}
    for _, row in context_data.iterrows():
        items = row['items']
        for i in range(len(items)):
            for j in range(i+1, len(items)):
                pair = tuple(sorted([items[i], items[j]]))
                item_pairs[pair] = item_pairs.get(pair, 0) + 1

    # Display top patterns
    top_patterns = sorted(item_pairs.items(), key=lambda x: x[1], reverse=True)[:10]
    print("Top patterns in this context:")
    for (item1, item2), count in top_patterns:
        support = count / len(context_data)
        print(f"  {{{item1}, {item2}}}: support={support:.3f}")

# Mine patterns for different contexts
contexts_to_analyze = [
    {'weather': 'rainy', 'time_of_day': 'evening', 'location': 'Nairobi'},
    {'weather': 'hot', 'time_of_day': 'afternoon', 'location': 'Mombasa'},
    {'weather': 'sunny', 'time_of_day': 'morning', 'location': 'Nakuru'}
]

for context in contexts_to_analyze:
    mine_context_aware_patterns(df, context)

# 4. Real-Time Alert System Simulation
print("\n" + "="*50)
print("4. REAL-TIME ALERT SYSTEM SIMULATION")
print("="*50)

class RealTimeAlertSystem:
    def __init__(self):
        self.unusual_patterns = {}

    def monitor_transaction(self, transaction, historical_norm):
        """Monitor transaction for unusual patterns"""
        items = set(transaction['items'])
        context = (transaction['weather'], transaction['time_of_day'], transaction['location'])

        # Check for unusual item combinations
        unusual_combinations = self._detect_unusual_combinations(items, context, historical_norm)

        if unusual_combinations:
            print(f"ALERT: Unusual pattern detected in transaction {transaction['transaction_id']}")
            print(f"   Context: {context}")
            print(f"   Items: {transaction['items']}")
            print(f"   Unusual combinations: {unusual_combinations}")

    def _detect_unusual_combinations(self, items, context, historical_norm):
        """Detect combinations that are unusual for given context"""
        unusual = []

        # Simplified logic - in reality, this would use statistical models
        if context[0] == 'rainy' and 'Ice Cream' in items and 'Cold Soda' in items:
            unusual.append("Cold items during rainy weather")

        if context[1] == 'morning' and 'Chicken' in items and 'Beef' in items:
            unusual.append("Multiple meats in morning")

        return unusual

# Simulate real-time monitoring
print("\nSimulating real-time anomaly detection...")
alert_system = RealTimeAlertSystem()

# Test with some transactions
test_transactions = [
    {'transaction_id': 'TEST_001', 'items': ['Ice Cream', 'Cold Soda', 'Maize Flour'],
     'weather': 'rainy', 'time_of_day': 'evening', 'location': 'Nairobi'},
    {'transaction_id': 'TEST_002', 'items': ['Chicken', 'Beef', 'Cooking Oil'],
     'weather': 'sunny', 'time_of_day': 'morning', 'location': 'Kisumu'}
]

for transaction in test_transactions:
    alert_system.monitor_transaction(transaction, historical_norm=None)

# 5. Business Insights
print("\n" + "="*50)
print("5. BUSINESS INSIGHTS & RECOMMENDATIONS")
print("="*50)

print("\n REAL-TIME ARM BENEFITS FOR KENYAN RETAIL:")
print("1. Dynamic Pricing: Adjust prices based on real-time demand patterns")
print("2. Personalized Marketing: Context-aware promotions via SMS/USSD")
print("3. Inventory Optimization: Real-time stock alerts for frequent combinations")
print("4. Customer Experience: Instant recommendations in shopping apps")

print("\n CONTEXT-AWARE STRATEGIES:")
print("• Rainy days: Promote umbrellas, hot beverages, soups")
print("• Morning rush: Breakfast bundles (bread, eggs, milk, chai)")
print("• Coastal regions: Coconut-based product promotions")
print("• Holiday seasons: Special meal packages")

print(f"\n Implementation ready for Kenyan retail with {len(df)} historical transactions")
print("Can integrate with mobile money platforms for seamless promotions")

=== KENYAN RETAIL REAL-TIME & CONTEXT-AWARE ARM DEMONSTRATION ===

1. Generating Kenyan Retail Dataset with Context...
Generated 5000 transactions
Sample transactions with context:
transaction_id                                                               items weather time_of_day location
     TXN_00001                   [Boots, Sugar, Tomatoes, Fresh Milk, Wheat Flour]    cold     evening   Nakuru
     TXN_00002 [Sugar, Eggs, Fresh Milk, Bread, Rice, Coffee, Wheat Flour, Cheese]   sunny     morning   Nakuru
     TXN_00003                               [Special Rice, Wheat Flour, Umbrella]   rainy   afternoon  Nairobi

2. REAL-TIME ARM SIMULATION

Simulating real-time transaction processing...

Testing real-time recommendations:
Context: Weather=rainy, Time=morning, Location=Nairobi
Current items: ['Maize Flour']
Recommended items:
  - Eggs (score: 8)
  - Bread (score: 8)
  - Chai Masala (score: 5)
  - Milk (score: 4)
  - Hot Chocolate (score: 3)

3. CONTEXT-AWARE PATTERN MINING

Mi